<a href="https://colab.research.google.com/github/philippwulff/behaviour_mining/blob/master/Feature_Space_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Theory:
--

Timeseries clustering is very complex, to reduce the complexity we extract metadata from the invervals and cluster them with the more traditional data point clustering methods

Imports
--

In [ ]:
!git clone https://github.com/philippwulff/behaviour_mining.git

Cloning into 'behaviour_mining'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 603 (delta 27), reused 0 (delta 0), pack-reused 556
Receiving objects: 100% (603/603), 492.46 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (181/181), done.
Checking out files: 100% (342/342), done.


In [ ]:
import os

import numpy as np
import pandas as pd

from scipy.signal import find_peaks

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get data

In [ ]:
def read_all(folder, nrows=None):
    """
    Read all .csv files from a directory and return the pandas Dataframes.
    """
    dfs = {}

    for root,dirs,files in os.walk(folder):
        for file in files:
            if file.endswith('.csv'):
                path = os.path.join(root, str(file))
                print('Creating DataFrame from {}'.format(path))
                df = pd.read_csv(path, nrows=nrows)
                dfs[path] = df

    return dfs

In [ ]:
# Reduce datacount so to not exceed RAM
nrows = None
path = 'behaviour_mining/data/BipedalWalker-v3/'
dfs = read_all(path, nrows)

Creating DataFrame from behaviour_mining/data/BipedalWalker-v3/acktr/acktr_BipedalWalker-v3.csv
Creating DataFrame from behaviour_mining/data/BipedalWalker-v3/td3/td3_BipedalWalker-v3.csv
Creating DataFrame from behaviour_mining/data/BipedalWalker-v3/trpo/trpo_BipedalWalker-v3.csv
Creating DataFrame from behaviour_mining/data/BipedalWalker-v3/ppo2/ppo2_BipedalWalker-v3.csv
Creating DataFrame from behaviour_mining/data/BipedalWalker-v3/a2c/a2c_BipedalWalker-v3.csv
Creating DataFrame from behaviour_mining/data/BipedalWalker-v3/ddpg/ddpg_BipedalWalker-v3.csv
Creating DataFrame from behaviour_mining/data/BipedalWalker-v3/sac/sac_BipedalWalker-v3.csv


In [ ]:
dfs['behaviour_mining/data/BipedalWalker-v3/ppo2/ppo2_BipedalWalker-v3.csv'].head()

,done,rewards,cumulative_reward,obs_0,obs_1,obs_2,obs_3,obs_4,obs_5,obs_6,obs_7,obs_8,obs_9,obs_10,obs_11,obs_12,obs_13,obs_14,obs_15,obs_16,obs_17,obs_18,obs_19,obs_20,obs_21,obs_22,obs_23,action_0,action_1,action_2,action_3
0,0.0,-0.152152,0.000000,-1.553152,0.058755,-2.111050,-0.150687,2.715659,0.936015,0.653329,-1.196846,1.073211,-3.279606,1.274947,-0.284248,-1.137195,1.535426,3.946275,3.969125,3.979726,3.973202,3.942354,3.876772,3.759370,3.565766,1.510411,0.112139,-0.473087,-1.000000,0.267759,-1.000000
1,0.0,-0.116266,-0.152152,-1.476143,0.548042,-2.000170,-0.556585,2.694441,-0.067539,0.361258,-1.196319,-0.931783,-3.029333,1.087949,-0.521497,-1.137885,-0.651285,3.890856,3.913426,3.923912,3.917504,3.887100,3.822428,3.706634,3.515666,1.510411,0.112139,-1.000000,-1.000000,0.271426,-0.428934
2,0.0,-0.155962,-0.268418,-1.393407,0.584033,-1.963198,-0.578523,2.501582,-0.995438,0.164044,-0.819076,-0.931783,-2.879289,0.738007,-0.285848,0.999536,-0.651285,3.832627,3.854902,3.865268,3.858983,3.829043,3.765327,3.651223,3.463026,1.510411,0.112139,-1.000000,-0.224652,1.000000,0.632351
3,0.0,-0.173834,-0.424380,-1.288625,0.738060,-1.942015,-1.047063,2.249776,-1.258509,-0.106864,-1.196320,-0.931783,-2.497924,1.714268,-0.415467,-0.698416,-0.651285,3.730530,3.752289,3.762444,3.756373,3.727250,3.665209,3.554070,3.370729,1.510411,0.112139,-0.879462,-1.000000,0.655143,-0.277381
4,0.0,-0.156413,-0.598214,-1.163460,0.881893,-1.908166,-1.135971,1.999996,-1.258509,-0.379750,-1.186960,-0.931783,-2.496081,0.046901,-0.158971,1.127991,-0.651285,3.621193,3.642399,3.652328,3.646487,3.618237,3.557990,3.450026,3.271887,1.510411,0.112139,-0.330683,-0.005658,-0.428750,0.846328


# Creating the meta data from trajectories

Using these meta data features:

```
    columns = ['mean', 'std', 'median', 'quant_25', 'quant_75', 'loc_max_mean', 
               'loc_max_num', 'loc_min_mean', 'loc_min_num', 'freq_iv', 'strongest_freq']
```



```
# Als Code formatiert
```

Helper functions
--

In [ ]:
def split_ser(ser, n=100):
    num_splits = int(len(ser)/n)
    splits = []
    for i in range(num_splits):
        splits.append(ser[i*n:(i+1)*n])

    return splits


def ser_to_meta(x, obs_name=None):
    mean = np.mean(x)
    std = np.std(x)
    median = np.median(sorted(x))
    quant_25 = np.quantile(sorted(x), 0.25)
    quant_75 = np.quantile(sorted(x), 0.75)

    # find all local maxima with a min distance of 10 between them
    loc_max, _ = find_peaks(x, distance=10)
    loc_max_mean = np.mean(x[loc_max])
    loc_max_num = len(loc_max)
    # find all local minima with a min distance of 10 between them
    loc_min, _ = find_peaks(x*-1, distance=10)
    loc_min_mean = np.mean(x[loc_min])
    loc_min_num = len(loc_min)

    # find n_bins strongest present frequencies with fast fourier transform
    n_bins = 5
    fft = np.fft.fft(x)
    T = 1 # sampling interval
    N = len(x)
    f = np.linspace(0, 1 / T, N)
    inds = np.argsort(np.abs(fft)[:N // 2])[-n_bins:] # get indexes of strongest freqs
    inds = inds[np.where(inds!=0)] # ignore the bin at f=0 (if this was a max) because this is the mean
    n_freq = f[inds]

    freq_iv = pd.Interval(min(n_freq), max(n_freq), closed='both')
    strongest_freq = f[inds[-1]] # np.argsort places hightest at the end

    sample = [mean,
              std,
              median,
              quant_25,
              quant_75,
              loc_max_mean,
              loc_max_num,
              loc_min_mean,
              loc_min_num,
              freq_iv,
              strongest_freq]
    columns = ['mean', 'std', 'median', 'quant_25', 'quant_75', 'loc_max_mean', 
               'loc_max_num', 'loc_min_mean', 'loc_min_num', 'freq_iv', 'strongest_freq']

    if obs_name:
        columns = [name+'_'+obs_name for name in columns]

    return sample, columns


def proc_ser(ser, model_name, obs_name=None, n=100):
    splits = split_ser(ser, n)
    metas = []
    columns = None
    for split in splits:
        meta, columns = ser_to_meta(split, obs_name)
        metas.append(meta)
        if not columns:
            columns = columns
    
    df_1 = pd.DataFrame([model_name]*len(metas), columns=['model'])
    df_2 = pd.DataFrame(metas, columns=columns)
    df = pd.concat([df_1,df_2], axis=1)
    return df

Concatenate meta data from all models for one observation to a single DataFrame
--

In [ ]:
def get_data(n_interval_lenght=100):
  obs_names = dfs['behaviour_mining/data/BipedalWalker-v3/ppo2/ppo2_BipedalWalker-v3.csv'].columns.values.tolist()
  obs_names.remove("done")
  obs_names.remove("cumulative_reward")
  model_names = [key.split('/')[-2] for key in dfs.keys()]

  df_all = pd.DataFrame()

  for model_df, model_name in zip(dfs.values(), model_names):
    model_meta_df = pd.DataFrame()
    for obs_name in obs_names:
      series = model_df[obs_name].values
      meta_df = proc_ser(series, model_name, obs_name=obs_name)
      if (obs_name!="rewards"):
        meta_df = meta_df.drop(columns="model")
      model_meta_df = pd.concat([model_meta_df, meta_df], axis=1)
    df_all = pd.concat([df_all, model_meta_df], axis=0)
  print("done with importing data")
  print (df_all)
  return df_all

In [ ]:
def remove_unstable_columns(df_all): 
  for obs_name in obs_names:
    #min_freqs, max_freqs = [], []
    #for iv in df_all['freq_iv']:
      #min_freqs.append(iv.left)
      #max_freqs.append(iv.right)
    #df_all['min_freq'+'_'+obs_name] = min_freqs
    #df_all['max_freq'+'_'+obs_name] = max_freqs
    df_all = df_all.drop('loc_max_mean'+'_'+obs_name, axis=1)
    df_all = df_all.drop('freq_iv'+'_'+obs_name, axis=1)
    df_all = df_all.drop('loc_min_mean'+'_'+obs_name, axis=1)
  df_all
  return df_all

If the trajectory of a model stays constantly 0, the np.mean() function will impute NaN as the mean value. 

In [ ]:
df_all.to_csv("test.csv", sep='\t')

In [ ]:
df_all = df_all.dropna(axis=0)

In [ ]:
df_all

,mean_rewards,std_rewards,median_rewards,quant_25_rewards,quant_75_rewards,loc_max_num_rewards,loc_min_num_rewards,strongest_freq_rewards,mean_obs_0,std_obs_0,median_obs_0,quant_25_obs_0,quant_75_obs_0,loc_max_num_obs_0,loc_min_num_obs_0,strongest_freq_obs_0,model,mean_obs_1,std_obs_1,median_obs_1,quant_25_obs_1,quant_75_obs_1,loc_max_num_obs_1,loc_min_num_obs_1,strongest_freq_obs_1,mean_obs_2,std_obs_2,median_obs_2,quant_25_obs_2,quant_75_obs_2,loc_max_num_obs_2,loc_min_num_obs_2,strongest_freq_obs_2,mean_obs_3,std_obs_3,median_obs_3,quant_25_obs_3,quant_75_obs_3,loc_max_num_obs_3,loc_min_num_obs_3,...,mean_obs_23,std_obs_23,median_obs_23,quant_25_obs_23,quant_75_obs_23,loc_max_num_obs_23,loc_min_num_obs_23,strongest_freq_obs_23,mean_action_0,std_action_0,median_action_0,quant_25_action_0,quant_75_action_0,loc_max_num_action_0,loc_min_num_action_0,strongest_freq_action_0,mean_action_1,std_action_1,median_action_1,quant_25_action_1,quant_75_action_1,loc_max_num_action_1,loc_min_num_action_1,strongest_freq_action_1,mean_action_2,std_action_2,median_action_2,quant_25_action_2,quant_75_action_2,loc_max_num_action_2,loc_min_num_action_2,strongest_freq_action_2,mean_action_3,std_action_3,median_action_3,quant_25_action_3,quant_75_action_3,loc_max_num_action_3,loc_min_num_action_3,strongest_freq_action_3
0,0.216653,0.197438,0.248870,0.106347,0.367415,8,7,0.010101,0.137795,0.375388,0.250719,-0.127700,0.444650,6,6,0.020202,acktr,0.072424,0.838593,0.014909,-0.503645,0.632804,8,6,0.131313,-0.952853,1.260560,-0.585035,-1.858036,0.195773,6,6,0.010101,-0.392201,1.362281,-0.122345,-0.674404,0.362754,7,6,...,0.044933,2.081668e-17,0.044933,0.044933,0.044933,0,0,0.040404,0.357207,0.734051,0.690881,-0.318348,1.0,7,7,0.070707,0.283222,0.863428,0.916696,-0.859429,1.0,6,6,0.070707,0.337964,0.782462,0.824500,-0.199557,1.0,5,6,0.070707,-0.162350,0.728219,-0.198746,-1.000000,0.414425,8,8,0.070707
1,0.462276,0.080798,0.464315,0.411331,0.504335,7,8,0.090909,-0.477323,0.383798,-0.433760,-0.671587,-0.246766,6,7,0.010101,acktr,-0.022875,0.919394,-0.162930,-0.646975,0.671602,7,7,0.070707,0.727173,0.186343,0.686081,0.592485,0.859332,6,6,0.010101,-0.060029,0.765073,0.065296,-0.474636,0.480049,6,6,...,0.044933,2.081668e-17,0.044933,0.044933,0.044933,0,0,0.040404,0.393849,0.705880,0.674436,-0.014643,1.0,7,7,0.060606,0.246752,0.875374,0.837821,-0.991566,1.0,6,6,0.060606,0.118104,0.872904,0.345465,-1.000000,1.0,7,6,0.060606,-0.076743,0.685093,0.029620,-0.823481,0.486944,8,7,0.060606
2,0.454991,0.082055,0.457093,0.391873,0.504265,8,6,0.060606,-0.320845,0.384294,-0.324781,-0.554932,0.021754,7,8,0.010101,acktr,-0.000709,0.855807,-0.186069,-0.626656,0.542613,7,8,0.070707,0.708997,0.193385,0.701957,0.567033,0.852338,7,7,0.010101,0.329925,0.810747,0.241379,-0.317887,1.013185,7,7,...,0.044933,2.081668e-17,0.044933,0.044933,0.044933,0,0,0.040404,0.367124,0.793469,1.000000,-0.307351,1.0,7,7,0.060606,0.244308,0.830249,0.643995,-0.644854,1.0,8,7,0.070707,0.040924,0.898273,0.183101,-1.000000,1.0,8,8,0.070707,-0.084708,0.715426,0.008722,-0.842191,0.573314,8,8,0.090909
3,0.486268,0.071129,0.485611,0.427694,0.537469,8,7,0.080808,-0.541695,0.373692,-0.524720,-0.830675,-0.289809,7,7,0.020202,acktr,-0.024694,0.763814,-0.187012,-0.643732,0.591851,6,8,0.060606,0.964932,0.173863,0.926456,0.864405,1.099814,7,6,0.060606,0.085052,0.481658,0.031561,-0.310943,0.415716,8,6,...,0.044933,2.081668e-17,0.044933,0.044933,0.044933,0,0,0.040404,0.401434,0.733477,0.916947,-0.276249,1.0,7,7,0.060606,0.121532,0.889704,0.374523,-1.000000,1.0,6,6,0.060606,-0.017190,0.893128,0.078984,-1.000000,1.0,7,7,0.060606,-0.043593,0.695282,-0.014716,-0.795065,0.537321,6,7,0.212121
4,0.487240,0.073009,0.483261,0.434056,0.538683,6,8,0.080808,-0.560289,0.400415,-0.443594,-0.767982,-0.285632,7,6,0.010101,acktr,0.061776,0.848690,-0.065700,-0.557438,0.798340,8,7,0.060606,0.886317,0.140645,0.867968,0.777957,0.965088,6,7,0.060606,0.123946,0.633493,0.036710,-0.316464,0.481091,7,7,...,0.044933,2.081668e-17,0.044933,0.0449

#clustering

In [ ]:
def cluster(df_all, n_clusters=7):
  df_without_model = df_all.drop(columns="model")
  n_clusters=7
  km = KMeans(n_clusters=n_clusters)
  km.fit(df_without_model)
  prediction = km.predict(df_without_model)
  occurance_dict={}
  for i in range(n_clusters):
    temp_dict={}
    for name in model_names:
      temp_dict[name]=0
    occurance_dict[i] = temp_dict
  for i in range(len(prediction)):
    occurance_dict[prediction[i]][df_all.iloc[i,0]] = occurance_dict[prediction[i]][df_all.iloc[i,0]]+1
  print("number of intervals that an algorithms occurs in the clusters")
  for i in range(n_clusters):
    print("cluster "+str(i)+": "+str(occurance_dict[i]))

In [ ]:
def get_data_and_cluster(n_clusters=7, n_interval_lenght=100):
  df_all = get_data(n_interval_lenght=n_interval_lenght)
  df_all = remove_unstable_columns(df_all)
  cluster(df_all, n_clusters=n_clusters)

#actual clusters

In [ ]:
get_data_and_cluster(7, n_interval_lenght=100)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars



done with importing data
     model  ...  strongest_freq_action_3
0    acktr  ...                 0.070707
1    acktr  ...                 0.060606
2    acktr  ...                 0.090909
3    acktr  ...                 0.212121
4    acktr  ...                 0.060606
..     ...  ...                      ...
995    sac  ...                 0.494949
996    sac  ...                 0.494949
997    sac  ...                 0.494949
998    sac  ...                 0.494949
999    sac  ...                 0.494949

[7000 rows x 320 columns]
number of intervals that an algorithms occurs in the clusters
cluster 0: {'acktr': 0, 'td3': 0, 'trpo': 0, 'ppo2': 0, 'a2c': 0, 'ddpg': 802, 'sac': 0}
cluster 1: {'acktr': 9, 'td3': 6, 'trpo': 885, 'ppo2': 0, 'a2c': 1, 'ddpg': 0, 'sac': 446}
cluster 2: {'acktr': 51, 'td3': 3, 'trpo': 89, 'ppo2': 903, 'a2c': 115, 'ddpg': 0, 'sac': 419}
cluster 3: {'acktr': 308, 'td3': 0, 'trpo': 1, 'ppo2': 84, 'a2c': 844, 'ddpg': 0, 'sac': 14}
cluster 4: {'acktr': 6, 't

In [ ]:
get_data_and_cluster(7, n_interval_lenght=30)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars



done with importing data
     model  ...  strongest_freq_action_3
0    acktr  ...                 0.070707
1    acktr  ...                 0.060606
2    acktr  ...                 0.090909
3    acktr  ...                 0.212121
4    acktr  ...                 0.060606
..     ...  ...                      ...
995    sac  ...                 0.494949
996    sac  ...                 0.494949
997    sac  ...                 0.494949
998    sac  ...                 0.494949
999    sac  ...                 0.494949

[7000 rows x 320 columns]
number of intervals that an algorithms occurs in the clusters
cluster 0: {'acktr': 55, 'td3': 0, 'trpo': 92, 'ppo2': 944, 'a2c': 230, 'ddpg': 0, 'sac': 409}
cluster 1: {'acktr': 0, 'td3': 7, 'trpo': 881, 'ppo2': 0, 'a2c': 7, 'ddpg': 0, 'sac': 410}
cluster 2: {'acktr': 933, 'td3': 0, 'trpo': 0, 'ppo2': 45, 'a2c': 742, 'ddpg': 0, 'sac': 5}
cluster 3: {'acktr': 0, 'td3': 0, 'trpo': 0, 'ppo2': 0, 'a2c': 0, 'ddpg': 802, 'sac': 0}
cluster 4: {'acktr': 6, 'td

In [ ]:
get_data_and_cluster(7, n_interval_lenght=15)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars



done with importing data
     model  ...  strongest_freq_action_3
0    acktr  ...                 0.070707
1    acktr  ...                 0.060606
2    acktr  ...                 0.090909
3    acktr  ...                 0.212121
4    acktr  ...                 0.060606
..     ...  ...                      ...
995    sac  ...                 0.494949
996    sac  ...                 0.494949
997    sac  ...                 0.494949
998    sac  ...                 0.494949
999    sac  ...                 0.494949

[7000 rows x 320 columns]
number of intervals that an algorithms occurs in the clusters
cluster 0: {'acktr': 2, 'td3': 0, 'trpo': 6, 'ppo2': 7, 'a2c': 11, 'ddpg': 193, 'sac': 0}
cluster 1: {'acktr': 0, 'td3': 7, 'trpo': 881, 'ppo2': 0, 'a2c': 7, 'ddpg': 0, 'sac': 409}
cluster 2: {'acktr': 3, 'td3': 988, 'trpo': 12, 'ppo2': 0, 'a2c': 1, 'ddpg': 0, 'sac': 176}
cluster 3: {'acktr': 0, 'td3': 0, 'trpo': 0, 'ppo2': 0, 'a2c': 0, 'ddpg': 802, 'sac': 0}
cluster 4: {'acktr': 933, 'td3':

In [ ]:
get_data_and_cluster(3, n_interval_lenght=100)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars



done with importing data
     model  ...  strongest_freq_action_3
0    acktr  ...                 0.070707
1    acktr  ...                 0.060606
2    acktr  ...                 0.090909
3    acktr  ...                 0.212121
4    acktr  ...                 0.060606
..     ...  ...                      ...
995    sac  ...                 0.494949
996    sac  ...                 0.494949
997    sac  ...                 0.494949
998    sac  ...                 0.494949
999    sac  ...                 0.494949

[7000 rows x 320 columns]
number of intervals that an algorithms occurs in the clusters
cluster 0: {'acktr': 933, 'td3': 0, 'trpo': 0, 'ppo2': 45, 'a2c': 742, 'ddpg': 0, 'sac': 5}
cluster 1: {'acktr': 3, 'td3': 988, 'trpo': 12, 'ppo2': 0, 'a2c': 1, 'ddpg': 0, 'sac': 170}
cluster 2: {'acktr': 0, 'td3': 7, 'trpo': 881, 'ppo2': 0, 'a2c': 7, 'ddpg': 0, 'sac': 409}
cluster 3: {'acktr': 3, 'td3': 0, 'trpo': 6, 'ppo2': 10, 'a2c': 12, 'ddpg': 191, 'sac': 0}
cluster 4: {'acktr': 0, 'td3

In [ ]:
get_data_and_cluster(3, n_interval_lenght=30)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars



done with importing data
     model  ...  strongest_freq_action_3
0    acktr  ...                 0.070707
1    acktr  ...                 0.060606
2    acktr  ...                 0.090909
3    acktr  ...                 0.212121
4    acktr  ...                 0.060606
..     ...  ...                      ...
995    sac  ...                 0.494949
996    sac  ...                 0.494949
997    sac  ...                 0.494949
998    sac  ...                 0.494949
999    sac  ...                 0.494949

[7000 rows x 320 columns]
number of intervals that an algorithms occurs in the clusters
cluster 0: {'acktr': 9, 'td3': 6, 'trpo': 885, 'ppo2': 0, 'a2c': 1, 'ddpg': 0, 'sac': 446}
cluster 1: {'acktr': 308, 'td3': 0, 'trpo': 1, 'ppo2': 84, 'a2c': 844, 'ddpg': 0, 'sac': 14}
cluster 2: {'acktr': 6, 'td3': 5, 'trpo': 9, 'ppo2': 4, 'a2c': 8, 'ddpg': 5, 'sac': 6}
cluster 3: {'acktr': 624, 'td3': 986, 'trpo': 10, 'ppo2': 2, 'a2c': 21, 'ddpg': 0, 'sac': 115}
cluster 4: {'acktr': 51, 'td

In [ ]:
get_data_and_cluster(3, n_interval_lenght=15)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars



done with importing data
     model  ...  strongest_freq_action_3
0    acktr  ...                 0.070707
1    acktr  ...                 0.060606
2    acktr  ...                 0.090909
3    acktr  ...                 0.212121
4    acktr  ...                 0.060606
..     ...  ...                      ...
995    sac  ...                 0.494949
996    sac  ...                 0.494949
997    sac  ...                 0.494949
998    sac  ...                 0.494949
999    sac  ...                 0.494949

[7000 rows x 320 columns]
number of intervals that an algorithms occurs in the clusters
cluster 0: {'acktr': 56, 'td3': 0, 'trpo': 92, 'ppo2': 944, 'a2c': 231, 'ddpg': 0, 'sac': 405}
cluster 1: {'acktr': 0, 'td3': 7, 'trpo': 881, 'ppo2': 0, 'a2c': 7, 'ddpg': 0, 'sac': 411}
cluster 2: {'acktr': 0, 'td3': 0, 'trpo': 0, 'ppo2': 0, 'a2c': 0, 'ddpg': 802, 'sac': 0}
cluster 3: {'acktr': 933, 'td3': 0, 'trpo': 0, 'ppo2': 45, 'a2c': 741, 'ddpg': 0, 'sac': 4}
cluster 4: {'acktr': 3, 'td